In [15]:
from gurobipy import Model, multidict, GRB,LinExpr
import pandas as pd

# df = pd.read_excel("data.xlsx")
df = pd.read_csv("data4.csv")
# Define the list of DMUs
# DMU = ["A", "B", "C", "D", "E"]

# Create dictionaries to store various information
E = {}
# val_p1, val_p2, val_p3, = {}, {}, {}, {}
val_s1, val_s2 = {}, {}
# slack_p1, slack_p2, slack_p3 = {}, {}, {}


processes = df['process'].unique().tolist()
processes.remove('total')  # Remove 'total' from the list of processes

x_columns = [col for col in df.columns if col.startswith('x')]


# Define the number of inputs (I) and outputs (O) for each DMU
I = len(x_columns)
O = len(processes)
print(I,O)
print(processes)
df

8 5
['1', '2', '3', '4', 'f']


,DMU,process,x1,x2,x3,x4,x5,x6,x7,x8,...,y3-O,y3-I,y4-O,y4-I,y5-O,y5-I,y6-O,y6-I,y7-O,yf
0,H1,total,500.0,1000,200.00,300.00,50.00,100.0,20.00,5.000,...,1080.0,NaN,1020.0,NaN,960.0,NaN,840.0,NaN,4.8,12000000.0
1,H1,1,200.0,400,50.00,100.00,10.00,20.0,5.00,1.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H1,2,100.0,200,50.00,50.00,10.00,20.0,5.00,1.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H1,3,100.0,200,50.00,100.00,20.00,40.0,5.00,2.000,...,1080.0,1020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H1,4,50.0,100,25.00,25.00,5.00,10.0,3.00,0.500,...,NaN,NaN,1020.0,960.0,NaN,NaN,NaN,NaN,NaN,NaN
5,H1,f,50.0,100,25.00,25.00,5.00,10.0,2.00,0.500,...,NaN,NaN,NaN,NaN,960.0,840.0,840.0,NaN,4.8,12000000.0
6,H2,total,450.0,900,180.00,270.00,45.00,90.0,18.00,4.500,...,972.0,NaN,918.0,NaN,864.0,NaN,756.0,NaN,4.7,10800000.0
7,H2,1,180.0,360,45.00,90.00,9.00,18.0,4.50,0.900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,H2,2,90.0,180,45.00,45.00,9.00,18.0,4.50,0.900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,H2,3,90.0,180,45.00,90.00,18.00,36.0,4.50,1.800,...,972.0,918.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Tot = df[df["process"] == "total"][["DMU", "x1", "x2", "x3"]].set_index("DMU")
Tot = df[df["process"]=="total"][["DMU"]+x_columns].set_index("DMU")
DMU = Tot.index.to_list()
Tot = Tot.to_dict()

In [17]:
procs = {}
PP = {}
val_p={}
slack_p={}
for process in processes:
    PP[process]={}
    val_p[process]={}
    slack_p[process]={}
    # print(process)
    if process == "f":
        # Define more data specific to each DMU (process 3)
        proc = df[df["process"] == process][["DMU"] + x_columns +[f"y{process}"]].set_index("DMU")
        proc.rename(columns={f"y{process}": "TotyO"}, inplace=True)

    else:
        proc = df[df["process"] == process][
            ["DMU"] + x_columns + [f"y{process}-O", f"y{process}-I"]
        ].set_index("DMU")
        
        proc.rename(columns={f"y{process}-O": "yO", f"y{process}-I": "yI"}, inplace=True)
        proc["TotyO"] = proc["yO"] + proc["yI"]
        # proc = proc.to_dict()
    # display(proc)
    proc = proc.to_dict()
    # procs.append(proc)
    procs[process]=proc

procs

{'1': {'x1': {'H1': 200.0,
   'H2': 180.0,
   'H3': 220.0,
   'H4': 160.0,
   'H5': 240.0,
   'H6': 140.0,
   'H7': 210.0,
   'H8': 190.0},
  'x2': {'H1': 400,
   'H2': 360,
   'H3': 440,
   'H4': 320,
   'H5': 480,
   'H6': 280,
   'H7': 420,
   'H8': 380},
  'x3': {'H1': 50.0,
   'H2': 45.0,
   'H3': 55.0,
   'H4': 40.0,
   'H5': 60.0,
   'H6': 35.0,
   'H7': 52.5,
   'H8': 47.5},
  'x4': {'H1': 100.0,
   'H2': 90.0,
   'H3': 110.0,
   'H4': 80.0,
   'H5': 120.0,
   'H6': 70.0,
   'H7': 105.0,
   'H8': 95.0},
  'x5': {'H1': 10.0,
   'H2': 9.0,
   'H3': 11.0,
   'H4': 8.0,
   'H5': 12.0,
   'H6': 7.0,
   'H7': 10.5,
   'H8': 9.5},
  'x6': {'H1': 20.0,
   'H2': 18.0,
   'H3': 22.0,
   'H4': 16.0,
   'H5': 24.0,
   'H6': 14.0,
   'H7': 21.0,
   'H8': 19.0},
  'x7': {'H1': 5.0,
   'H2': 4.5,
   'H3': 5.5,
   'H4': 4.0,
   'H5': 6.0,
   'H6': 3.5,
   'H7': 5.25,
   'H8': 4.75},
  'x8': {'H1': 1.0,
   'H2': 0.9,
   'H3': 1.1,
   'H4': 0.8,
   'H5': 1.2,
   'H6': 0.7,
   'H7': 1.05,
   'H8'

In [18]:

v, u = {}, {}
for k in DMU:
    # Create the Gurobi model
    m = Model("network_DEA")

    # Define decision variables
    v = {x: m.addVar(vtype=GRB.CONTINUOUS, name=f"v_{x}") for x in x_columns}
    u = {key: m.addVar(vtype=GRB.CONTINUOUS, name=f"u_{key}") for key,val in procs.items()}

    # Update the model
    m.update()

    # Set the objective function to maximize efficiency
    # m.setObjective(
    #     u[0] * proc1["yO"][k] + u[1] * proc2["yO"][k] + u[2] * proc3["TotyO"][k],
    #     GRB.MAXIMIZE,
    # )
    obj_expression = LinExpr()
    # for i, proc in enumerate(procs):
    for key, proc in procs.items():
        # print(key,proc)
        attr = proc.get("yO", proc.get("TotyO"))
        obj_expression += u[key] * attr[k]
    m.setObjective(obj_expression, GRB.MAXIMIZE)

    # # Add constraints
    # m.addConstr(v[0] * Tot["x1"][k] + v[1] * Tot["x2"][k] + v[2] * Tot["x3"][k] == 1)
    const_expression = LinExpr()
    for x in x_columns:
        const_expression += v[x] * Tot[x][k]
    m.addConstr(const_expression == 1)

    # for j in DMU:
    #     P1[j] = m.addConstr(
    #         u[0] * proc1["TotyO"][j]
    #         - (v[0] * proc1["x1"][j] + v[1] * proc1["x2"][j] + v[2] * proc1["x3"][j])
    #         <= 0
    #     )
    #     P2[j] = m.addConstr(
    #         u[1] * proc2["TotyO"][j]
    #         - (v[0] * proc2["x1"][j] + v[1] * proc2["x2"][j] + v[2] * proc2["x3"][j])
    #         <= 0
    #     )
    #     P3[j] = m.addConstr(
    #         u[2] * proc3["TotyO"][j]
    #         - (
    #             v[0] * proc3["x1"][j]
    #             + v[1] * proc3["x2"][j]
    #             + v[2] * proc3["x3"][j]
    #             + u[0] * proc1["yI"][j]
    #             + u[1] * proc2["yI"][j]
    #         )
    #         <= 0
    #     )

    for j in DMU:
        const_expression_yI = LinExpr()
        # for p, proc in enumerate(procs):
        for key, proc in procs.items():

            const_expression = LinExpr()
            const_expression += u[key] * proc["TotyO"][j]
            # print(const_expression)
            # const_expression = LinExpr()
            # for i, X in enumerate(x_columns):
            for x in x_columns:
                const_expression -= v[x] * proc[x][j]
                # print(const_expression)
            # print((const_expression))

            if proc.get("yI") is not None:
                const_expression_yI += u[key] * proc["yI"][j]
            else:
                const_expression -= const_expression_yI
            # print(const_expression)
            # print(key,proc)
            PP[key][j] = m.addConstr(const_expression <= 0)

    # # Optimize the model
    m.optimize()

    # # Store efficiency values in the dictionary
    # # E[k] = f"The efficiency of DMU {k}: {m.objVal}"
    # E[k] = f"efficiency of DMU {k} is: {m.objVal}"
    E[k] = m.objVal

    u_sol = m.getAttr("x", u)
    v_sol = m.getAttr("x", v)

    # Calculate process and stage efficiencies
    # for p, proc in enumerate(procs):
    denominator2 = 0
    for key, proc in procs.items():
        numerator = u_sol[key] * proc["TotyO"][k]
        denominator = 0
        # denominator2 = 0
        # for i, X in enumerate(x_columns):
        for x in x_columns:
            denominator += v_sol[x] * proc[x][k]
        if proc.get("yI") is not None:
            denominator2 += u_sol[key] * proc["yI"][k]
        else:
            denominator +=denominator2
        E_proc = numerator / denominator
        # print(p+1,E_proc,k)
        # val_p[key]={}
        # val_p[p+1][k]=1
        val_p[key][k] = f"The efficiency of process {key} of DMU {k}: {E_proc:.4g}"

    # # Calculate process and stage efficiencies
    # E1 = (
    #     u_sol[0]
    #     * proc1["TotyO"][k]
    #     / (
    #         v_sol[0] * proc1["x1"][k]
    #         + v_sol[1] * proc1["x2"][k]
    #         + v_sol[2] * proc1["x3"][k]
    #     )
    # )

    # E2 = (
    #     u_sol[1]
    #     * proc2["TotyO"][k]
    #     / (
    #         v_sol[0] * proc2["x1"][k]
    #         + v_sol[1] * proc2["x2"][k]
    #         + v_sol[2] * proc2["x3"][k]
    #     )
    # )
    # E3 = (
    #     u_sol[2]
    #     * proc3["TotyO"][k]
    #     / (
    #         v_sol[0] * proc3["x1"][k]
    #         + v_sol[1] * proc3["x2"][k]
    #         + v_sol[2] * proc3["x3"][k]
    #         + u_sol[0] * proc1["yI"][k]
    #         + u_sol[1] * proc2["yI"][k]
    #     )
    # )


    # stage1 = (
    #     u_sol[0] * proc1["TotyO"][k]
    #     + u_sol[1] * proc2["TotyO"][k]
    #     + v_sol[0] * proc3["x1"][k]
    #     + v_sol[1] * proc3["x2"][k]
    #     + v_sol[2] * proc3["x3"][k]
    # ) / (v_sol[0] * Tot["x1"][k] + v_sol[1] * Tot["x2"][k] + v_sol[2] * Tot["x3"][k])

    # stage2 = (
    #     u_sol[0] * proc1["yO"][k]
    #     + u_sol[1] * proc2["yO"][k]
    #     + u_sol[2] * proc3["TotyO"][k]
    # ) / (
    #     u_sol[0] * proc1["TotyO"][k]
    #     + u_sol[1] * proc2["TotyO"][k]
    #     + v_sol[0] * proc3["x1"][k]
    #     + v_sol[1] * proc3["x2"][k]
    #     + v_sol[2] * proc3["x3"][k]
    # )

    # Calculate stage efficiencies
    stage1_num = sum(u_sol[key] * proc["TotyO"][k] for key, proc in list(procs.items())[:-1])
    stage1_num += sum(v_sol[x] * procs["f"][x][k] for x in x_columns)
    stage1_den = sum(v_sol[x] * Tot[x][k]  for x in x_columns)
    stage1 = stage1_num / stage1_den

    stage2_num = sum(u_sol[key] * proc.get("yO", proc["TotyO"])[k] for key, proc in procs.items())
    stage2_den = stage1_num
    stage2 = stage2_num / stage2_den

    val_s1[k] = f"The efficiency of stage 1 of DMU {k}: {stage1:.4g}"
    val_s2[k] = f"The efficiency of stage 2 of DMU {k}: {stage2:.4g}"


    # # Store process and stage efficiency values
    # val_p1[k] = f"The efficiency of process 1 of DMU {k}: {E1:.4g}"
    # val_p2[k] = f"The efficiency of process 2 of DMU {k}: {E2:.4g}"
    # val_p3[k] = f"The efficiency of process 3 of DMU {k}: {E3:.4g}"
    # val_s1[k] = f"The efficiency of stage 1 of DMU {k}: {stage1:.4g}"
    # val_s2[k] = f"The efficiency of stage 2 of DMU {k}: {stage2:.4g}"

    # # Calculate and store inefficiency values
    # process1_slack = m.getAttr("slack", P1)
    # slack_p1[k] = f"The inefficiency of process 1 of DMU {k}: {process1_slack[k]:.4g}"
    # process2_slack = m.getAttr("slack", P2)
    # slack_p2[k] = f"The inefficiency of process 2 of DMU {k}: {process2_slack[k]:.4g}"
    # process3_slack = m.getAttr("slack", P3)
    # slack_p3[k] = f"The inefficiency of process 3 of DMU {k}: {process3_slack[k]:.4g}"

    # Calculate and store inefficiency values
    for key, proc in procs.items():
        # slack_p[key]={}
        # print(PP[key])
        process_slack = m.getAttr("slack", PP[key])
        # print(key,k,process_slack[k])
        slack_p[key][k] = f"The inefficiency of process {key} of DMU {k}: {process_slack[k]:.4g}"


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.0.0 24A335)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 41 rows, 13 columns and 400 nonzeros
Model fingerprint: 0x7faf18f4
Coefficient statistics:
  Matrix range     [3e-01, 1e+07]
  Objective range  [1e+03, 1e+07]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 10 rows and 3 columns
Presolve time: 0.00s
Presolved: 31 rows, 10 columns, 209 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2011638e+04   3.875149e+00   0.000000e+00      0s
      17    1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 17 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.000000000e+00
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.0.0 24A335)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 t

In [19]:
# Display the results for each DMU
for k in DMU:
    print(f"efficiency of DMU {k} is: {E[k]}")

    for key, proc in procs.items():
        print(val_p[key][k])

    print(val_s1[k])
    print(val_s2[k])

    for key, proc in procs.items():
        print(slack_p[key][k])
    print("")

efficiency of DMU H1 is: 0.9999999999999999
The efficiency of process 1 of DMU H1: 1
The efficiency of process 2 of DMU H1: 1
The efficiency of process 3 of DMU H1: 1
The efficiency of process 4 of DMU H1: 1
The efficiency of process f of DMU H1: 1
The efficiency of stage 1 of DMU H1: 1
The efficiency of stage 2 of DMU H1: 1
The inefficiency of process 1 of DMU H1: 0
The inefficiency of process 2 of DMU H1: 0
The inefficiency of process 3 of DMU H1: 0
The inefficiency of process 4 of DMU H1: 0
The inefficiency of process f of DMU H1: 0

efficiency of DMU H2 is: 1.0
The efficiency of process 1 of DMU H2: 1
The efficiency of process 2 of DMU H2: 1
The efficiency of process 3 of DMU H2: 1
The efficiency of process 4 of DMU H2: 1
The efficiency of process f of DMU H2: 1
The efficiency of stage 1 of DMU H2: 1
The efficiency of stage 2 of DMU H2: 1
The inefficiency of process 1 of DMU H2: 0
The inefficiency of process 2 of DMU H2: 0
The inefficiency of process 3 of DMU H2: 0
The inefficiency

In [20]:
dict(sorted(E.items(), key=lambda item: item[1],reverse=True))

{'H2': 1.0,
 'H1': 0.9999999999999999,
 'H3': 0.6035069491911598,
 'H4': 0.6035069491911598,
 'H5': 0.5657877648667121,
 'H7': 0.4905012820651918,
 'H6': 0.49043546094673907,
 'H8': 0.415078901400231}